In [18]:
# fast correlation based filter python implementation (FCBF)
# https://www.aaai.org/Papers/ICML/2003/ICML03-111.pdf

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from math import log, e
from scipy import stats

# save load_iris() sklearn dataset to iris
iris = load_iris()

df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])


In [4]:
df.head()

# shows we have 4 features.. low dimensional but will be fine for now

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [19]:
def entropy1(labels, base=None):
    value, counts = np.unique(labels, return_counts=True)
    return entropy(counts, base=base)

# entropy calculation
def entropy(x, base=2):

    if len(x) <= 1:
        return 0

    value, counts = np.unique(x, return_counts=True)
    probs = counts / len(x)
    n_classes = np.count_nonzero(probs)

    if n_classes <= 1:
        return 0

    ent = 0.

    base = e if base is None else base
    for i in probs:
        ent -= i * log(i, base)

    return ent


In [ ]:
# information gain

In [17]:
# entropy functions all seem to output different things?
# also, is running a np.unique() the right thing to do?

print(entropy(df['sepal length (cm)'], base=None))
print(entropy1(df['sepal length (cm)']))
print(stats.entropy(df['sepal length (cm)']))

3.3423682425704615
2.1194740021942584
5.000731141075856
